In [ ]:
from flask import Flask, render_template, redirect, url_for, session, request, flash
from flask_sqlalchemy import SQLAlchemy
from flask_wtf import FlaskForm
from wtforms import StringField, PasswordField, SubmitField, BooleanField, TextAreaField
from wtforms.validators import InputRequired, Email, Length, ValidationError
from wtforms.widgets import TextArea
from flask_login import UserMixin, LoginManager, login_user, login_required, logout_user, current_user
from flask_bcrypt import Bcrypt
import pandas as pd
from db_schema import *
import requests
import json
import time
import logging
from logging import FileHandler, WARNING

app = Flask(__name__)
    
file_handler = FileHandler('errorlog.txt')
file_handler.setLevel(WARNING)
app.logger.addHandler(file_handler)

db = SQLAlchemy(app)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///database.db'
app.config['SECRET_KEY'] = 'asecretkey'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

    
bcrypt = Bcrypt(app)

login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = "login"

@login_manager.user_loader
def load_user(user_id):
    return User.query.get(int(user_id))

################################################################################################################

# DEFINE FORMS

################################################################################################################


class RegisterForm(FlaskForm):
    email = StringField("Email", validators=[InputRequired(), Email(message="Invalid Email"), Length(max=50)], render_kw={"placeholder": "example@gmail.com"})
    username = StringField("Username", validators=[InputRequired(), Length(min=4, max=15)], render_kw={"placeholder": "Username"})
    password = PasswordField("Password", validators=[InputRequired(), Length(min=4, max=15)], render_kw={"placeholder": "********"})
    submit = SubmitField("Sign Up")

    def validate_username(self, username):
        existing_user_username = User.query.filter_by(username=username.data).first()
        if existing_user_username:
            raise ValidationError("That username already exists. Please choose a different one.")

    def validate_email(self, email):
        existing_user_email = User.query.filter_by(email=email.data).first()
        if existing_user_email:
            raise ValidationError("That email address belongs to different user. Please choose a different one.")


class LoginForm(FlaskForm):
    username = StringField("Username", validators=[InputRequired(), Length(max=15)], render_kw={"placeholder": "Username"})
    password = PasswordField("Password", validators=[InputRequired(), Length(max=50)], render_kw={"placeholder":  "Password"})
    submit = SubmitField("Login")

class NewNoteForm(FlaskForm):
    title = StringField("Title", validators=[InputRequired(), Length(max=25)], render_kw={"placeholder": "Title"})
    note_body = StringField(u'Text', widget=TextArea())
    submit = SubmitField("Add Note")

    
################################################################################################################

# FUNDAMENTAL ROUTES

################################################################################################################


@app.route('/index')
@app.route('/')
def home():
    return render_template('index.html', title='首页')


@app.route('/register', methods=['GET','POST'])
def register():
    form = RegisterForm()
    if form.validate_on_submit():
        hashed_password = bcrypt.generate_password_hash(form.password.data)
        new_user = User(username=form.username.data, email=form.email.data, password=hashed_password)
        db.session.add(new_user)
        db.session.commit()
        return redirect(url_for('login'))
    return render_template('register.html', title='注册', form=form)

@app.route('/login', methods=['GET','POST'])
def login():
    form = LoginForm()
    if form.validate_on_submit():
        user = User.query.filter_by(username=form.username.data).first()
        if user:
            if bcrypt.check_password_hash(user.password, form.password.data):
                login_user(user)
                return redirect(url_for("my_notes"))
        flash("User does not exist, or invalid username or password.")
    return render_template('login.html', title="登录", form=form)

@app.route('/logout', methods=["GET","POST"])
@login_required
def logout():
    session.clear()
    logout_user()
    return redirect(url_for('login'))   


@app.route("/historical")
def historical():    
    sqlalchemyObj = db.engine.execute('select * from venues WHERE venues.style == "历史建筑" ORDER BY id')
    venues = []
    for i in sqlalchemyObj:
        venues.append(i)
    #print(venues)
    
    dataset = []
    dict={}
    for i in venues:
        dict['Name'] = i[1]
        dict['Lat'] = i[6]
        dict['Lng'] = i[5]
        dict['Img_url'] = i[7]
        dict['Add'] = i[3]        
        dict['Trans'] = i[4]
        dict['Desc'] = i[8]
        #print(i)
        #print(dict)
        dataset.append(dict.copy()) #markers.append(fld.copy())
    #print(dataset)

    return render_template('historical.html', title='复古老上海', entries = dataset)

@app.route("/homestay")
def homestay():    
    sqlalchemyObj = db.engine.execute('select * from venues WHERE venues.style == "特色民宿" ORDER BY id')
    venues = []
    for i in sqlalchemyObj:
        venues.append(i)
    dataset = []
    dict={}
    for i in venues:
        dict['Name'] = i[1]
        dict['Lat'] = i[6]
        dict['Lng'] = i[5]
        dict['Img_url'] = i[7]
        dict['Add'] = i[3]        
        dict['Trans'] = i[4]
        dict['Desc'] = i[8]
        dataset.append(dict.copy()) #markers.append(fld.copy())
    return render_template('homestay.html', title='个性限定款', entries = dataset)

@app.route("/homestay-demo")
def homestay_demo():    
    sqlalchemyObj = db.engine.execute('select * from venues WHERE venues.name == "八十八亩田" ORDER BY id')
    venues = []
    for i in sqlalchemyObj:
        venues.append(i)
    dataset = []
    dict={}
    for i in venues:
        dict['Name'] = i[1]
        dict['Lat'] = i[6]
        dict['Lng'] = i[5]
        dict['Img_url'] = i[7]
        dict['Add'] = i[3]        
        dict['Trans'] = i[4]
        dict['Desc'] = i[8]
        dataset.append(dict.copy()) #markers.append(fld.copy())
    return render_template('homestay_demo.html', title='demo', entries = dataset)

@app.route("/museum")
def museum():    
    sqlalchemyObj = db.engine.execute('select * from venues WHERE venues.style == "艺术场馆" ORDER BY id')
    venues = []
    for i in sqlalchemyObj:
        venues.append(i)
    dataset = []
    dict={}
    for i in venues:
        dict['Name'] = i[1]
        dict['Lat'] = i[6]
        dict['Lng'] = i[5]
        dict['Img_url'] = i[7]
        dict['Add'] = i[3]        
        dict['Trans'] = i[4]
        dict['Desc'] = i[8]
        dataset.append(dict.copy()) 
    return render_template('museum.html', title='摩登艺术派', entries = dataset)

@app.route("/park")
def park():    
    sqlalchemyObj = db.engine.execute('select * from venues WHERE venues.style == "公园湿地" ORDER BY id')
    venues = []
    for i in sqlalchemyObj:
        venues.append(i)
    dataset = []
    dict={}
    for i in venues:
        dict['Name'] = i[1]
        dict['Lat'] = i[6]
        dict['Lng'] = i[5]
        dict['Img_url'] = i[7]
        dict['Add'] = i[3]        
        dict['Trans'] = i[4]
        dict['Desc'] = i[8]
        dataset.append(dict.copy()) 
    return render_template('park.html', title='自然田园风', entries = dataset)

@app.route("/pudong")
def pudong():    
    sqlalchemyObj = db.engine.execute('select * from venues WHERE venues.district IN ("浦东新区", "崇明区") ORDER BY id')
    venues = []
    for i in sqlalchemyObj:
        venues.append(i)
    dataset = []
    dict={}
    for i in venues:
        dict['Name'] = i[1]
        dict['Lat'] = i[6]
        dict['Lng'] = i[5]
        dict['Img_url'] = i[7]
        dict['Add'] = i[3]        
        dict['Trans'] = i[4]
        dict['Desc'] = i[8]
        dataset.append(dict.copy()) 
    return render_template('pudong.html', title='浦东', entries = dataset)

@app.route("/puxi")
def puxi():    
    sqlalchemyObj = db.engine.execute('select * from venues WHERE venues.district IN ("静安区", "黄浦区","杨浦区","虹口区","长宁区","徐汇区","普陀区","闵行区","宝山区","嘉定区","青浦区","松江区","奉贤区","金山区") ORDER BY id')
    venues = []
    for i in sqlalchemyObj:
        venues.append(i)
    dataset = []
    dict={}
    for i in venues:
        dict['Name'] = i[1]
        dict['Lat'] = i[6]
        dict['Lng'] = i[5]
        dict['Img_url'] = i[7]
        dict['Add'] = i[3]        
        dict['Trans'] = i[4]
        dict['Desc'] = i[8]
        dataset.append(dict.copy()) 
    return render_template('puxi.html', title='浦西', entries = dataset)

@app.route("/photographer")
def photographer():    
    sqlalchemyObj = db.engine.execute('select * from photographer ORDER BY id')
    photographers = []
    for i in sqlalchemyObj:
        photographers.append(i)
    #print(photographers)
    
    dataset = []
    dict={}
    for i in photographers:
        dict['Name'] = i[1]
        dict['Fee'] = i[4]
        dict['Photographer'] = i[6]
        dict['Url'] = i[2]
        dict['Img_url'] = i[3]        
        #print(i)
        #print(dict)
        dataset.append(dict.copy()) 
    #print(dataset)

    return render_template('photographer.html', title='摄影师', dataset=dataset)

@app.route("/weather")
def weather():  
    lat = 31.2222
    lon = 121.4581 
    api_key = "6a2e6f4934f3a105ea5b7bd15cf394ce"
    url = "https://api.openweathermap.org/data/2.5/onecall?lat=%s&lon=%s&appid=%s&units=metric" % (lat, lon, api_key)

    response = requests.get(url)
    data = json.loads(response.text)
    currentSunrise = data["current"]["sunrise"]
    currentSunset = data["current"]["sunset"]
    currentTemp = data["current"]["temp"]
    currentHumidity = data["current"]["humidity"]
    description = data["current"]["weather"][0]["description"]
    
    time_local_sunrise = time.localtime(currentSunrise)
    time_local_sunset = time.localtime(currentSunset)

    sunrise = time.strftime("%Y-%m-%d %H:%M:%S",time_local_sunrise)
    sunset = time.strftime("%Y-%m-%d %H:%M:%S",time_local_sunset)
    
    print(sunrise)
    print(sunset)
    
    temp1_min = data["daily"][0]["temp"]["min"]
    temp1_max = data["daily"][0]["temp"]["max"]
    description1 = data["daily"][0]["weather"][0]["description"]

    temp2_min = data["daily"][1]["temp"]["min"]
    temp2_max = data["daily"][1]["temp"]["max"]
    description2 = data["daily"][1]["weather"][0]["description"]

    temp3_min = data["daily"][2]["temp"]["min"]
    temp3_max = data["daily"][2]["temp"]["max"]
    description3 = data["daily"][2]["weather"][0]["description"]
    
    return render_template('weather.html', title='天气',description=description,
                           currentTemp=currentTemp,currentHumidity=currentHumidity,
                          sunrise=sunrise,sunset=sunset,
                           temp1_min=temp1_min,temp1_max=temp1_max,description1=description1,
                          temp2_min=temp2_min,temp2_max=temp2_max,description2=description2,
                          temp3_min=temp3_min,temp3_max=temp3_max,description3=description3)



@app.route("/guidance")
def guidance():  
    return render_template('guidance.html', title='Guidance')


################################################################################################################

# NOTES ROUTES

################################################################################################################

@app.route('/all-notes', methods=['GET','POST'])
def all_notes():
    notes = Note.query.order_by(Note.date_created.desc()).all()   
    return render_template('all_notes.html', notes=notes, title='找灵感')

@app.route('/my-notes', methods=['GET','POST'])
@login_required
def my_notes():    
    if current_user.username == 'admin':
        notes = Note.query.order_by(Note.date_created.desc()).all()
    else:
        notes = Note.query.filter_by(note_writer=str(current_user)).order_by(Note.date_created.desc()).all()
    return render_template('my_notes.html', notes=notes, title='我的')

@app.route('/new-note', methods=['GET','POST'])
@login_required
def new_note():
    form = NewNoteForm()
    if form.validate_on_submit():
        new_note = Note(title=form.title.data, note_body=form.note_body.data, note_writer=str(current_user), author=current_user.username)
        db.session.add(new_note)
        db.session.commit()
        return redirect(url_for('my_notes'))
    return render_template('new_note.html', title='新建', form=form)

@app.route('/delete-note/<int:note_id>', methods=['GET',"POST"])
@login_required
def delete_note(note_id):
    note = Note.query.get_or_404(note_id)
    db.session.delete(note)
    db.session.commit()
    return redirect(url_for('my_notes'))

@app.route('/update-note/<int:note_id>', methods=['GET',"POST"])
@login_required
def update_note(note_id):
    update_note = Note.query.filter_by(id=note_id).first()
    form = NewNoteForm()
    form.title.data = update_note.title
    form.note_body.data = update_note.note_body
    if form.validate_on_submit():
        update_note.title = request.form.get('title')
        update_note.note_body = request.form.get('note_body')
        db.session.commit()
        return redirect(url_for('my_notes'))
    return render_template('update_note.html', title='更新', form=form)

################################################################################################################

# ERROR HANDLERS

################################################################################################################

@app.errorhandler(404)
def page_not_found(e):
    return render_template('404.html'), 404

################################################################################################################

# FINALLY, RUN!

################################################################################################################
    
if __name__ == '__main__':
    app.run('localhost', 9004)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


/Users/mzread/opt/anaconda3/lib/python3.8/site-packages/flask_sqlalchemy/__init__.py:851: UserWarning: Neither SQLALCHEMY_DATABASE_URI nor SQLALCHEMY_BINDS is set. Defaulting SQLALCHEMY_DATABASE_URI to "sqlite:///:memory:".
  warnings.warn(
/Users/mzread/opt/anaconda3/lib/python3.8/site-packages/flask_sqlalchemy/__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(
/Users/mzread/opt/anaconda3/lib/python3.8/site-packages/flask_sqlalchemy/__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(
 * Running on http://localhost:9004/ (Press CTRL+C to quit)
127.0.0.1 - - [13/Dec/2021 22:08:58] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/202

2021-12-13 06:44:01
2021-12-13 16:52:39


127.0.0.1 - - [13/Dec/2021 22:10:19] "GET /all-notes HTTP/1.1" 200 -
